# チームいわな

最終的な精度　：RMSE 3251（40位 / 486チーム）  
モデル作成手法：勾配ブースティング

作成した主な説明変数  
・同一対戦カード別過去の観客動員数平均  
・試合直前の順位  
・試合直前の連勝数  
・2014年World Cup出場選手数  
・対戦チームの年棒総和  
・天気を「晴」「雨」に統一  
・曜日を「休日」「平日」に統一

・苦労した点  
①過去の観客動員数平均を計算するときに  
　平日や雨の影響で平均値がぶれる。  
②セレッソ大阪などは観客動員数が増加傾向。  
　ただ、年平均成長率（CAGR）を入れると過学習して精度悪化。  
③validationを2013/08以降のデータでやりたいが、学習にも使いたい。  
　いったん2013/08以降で精度確かめて、全データで学習しなおすのはやり方として微妙？  
　そもそも勾配ブースティングを少ないサンプルにやるのは筋が悪い？　
 

In [ ]:
import os
import copy
import math
import operator
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
from functools import reduce
from datetime import datetime
from IPython.core.display import display

import requests
import unicodedata
from bs4 import BeautifulSoup

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns",  200)
pd.set_option("display.max_rows",     2000)
pd.set_option("display.max_colwidth", 2000)
%matplotlib inline

In [ ]:
region_pref = {"北海道":["北海道"],
               "東北"  :["青森県","岩手県","秋田県","宮城県","山形県","福島県"],
               "関東"  :["茨城県","栃木県","群馬県","埼玉県","千葉県","東京都","神奈川県"],
               "中部"  :["新潟県","富山県","石川県","福井県","山梨県","長野県","岐阜県","静岡県","愛知県"],
               "近畿"  :["三重県","滋賀県","奈良県","和歌山県","京都府","大阪府","兵庫県"],
               "中国"  :["岡山県","広島県","鳥取県","島根県","山口県"],
               "四国"  :["香川県","徳島県","愛媛県","高知県"],
               "九州"  :["福岡県","佐賀県","長崎県","大分県","熊本県","宮崎県","鹿児島県","沖縄県"]}
near_region = {"北海道" : ["東海"],
               "東北"   : ["北海道","関東"],
               "関東"   : ["東北","中部"],
               "中部"   : ["関東","近畿"],
               "近畿"   : ["東海","中国"],
               "中国"   : ["近畿","四国"],
               "四国"   : ["中国","九州"],
               "九州"   : ["中国","四国"]}

In [ ]:
# カテゴリデータのダミー変数化
def dummies(df, cols):
    df_droped  = copy.deepcopy(df.drop(cols, axis=1)).reset_index(drop=True)
    df_dummies = df.reset_index(drop=True)
    df_dummies = pd.get_dummies(df_dummies[cols], drop_first=True)
    return pd.merge(df_droped, df_dummies, left_index=True, right_index=True)

# 数値データの標準化
def standardization(df, cols, df_test=None):
    mean     = df[cols].mean()
    std      = df[cols].std()
    cols_std = [col + "_std" for col in cols]
    df_std   = copy.deepcopy(df)
    df_std[cols_std] = df_std[cols].apply(lambda x: (x - mean[x.name]) / std[x.name])
    df_test_std      = copy.deepcopy(df_test)
    if df_test is not None:
        df_test_std[cols_std] = df_test_std[cols].apply(lambda x: (x - mean[x.name]) / std[x.name])
    return df_std, df_test_std

def re_weather(df):
    df_weather = copy.deepcopy(df)
    df_weather.weather = df_weather.weather.apply(lambda x: x if x.find("のち")==-1 else x.split("のち")[1])
    df_weather.weather = df_weather.weather.apply(lambda x: x.split("一時")[0])
    df_weather.weather = df_weather.weather.apply(lambda x: x.split("時々")[0])
    df_weather.weather = df_weather.weather.apply(lambda x: "晴" if x in ["屋内","曇"] else x)
    df_weather.weather = df_weather.weather.apply(lambda x: "雨" if x in ["雷雨","雪","霧"] else x.replace(" ",""))
    return df_weather

def re_weekday(df):
    public_holiday = ["2011年1月1日","2011年1月10日","2011年2月11日","2011年3月21日","2011年4月29日",
                      "2011年5月3日","2011年5月4日","2011年5月5日","2011年7月18日","2011年9月19日",
                      "2011年9月23日","2011年10月10日","2011年11月3日","2011年11月23日","2011年12月23日"]
    df_weekday = copy.deepcopy(df)
    df_weekday.weekday = df_weekday.apply(lambda x: "休" if -1 < x.gameday.find("祝") else x.weekday, axis=1)
    df_weekday.weekday = df_weekday.apply(lambda x: "休" if -1 < x.gameday.find("休") else x.weekday, axis=1)
    df_weekday.weekday = df_weekday.apply(lambda x: "休" if x.gameday in public_holiday else x.weekday, axis=1)
    df_weekday.loc[ df_weekday.weekday.isin(["土","日"]), "weekday"] = "休"
    df_weekday.loc[~df_weekday.weekday.isin(["休"]),      "weekday"] = "平"
    return df_weekday

In [ ]:
path    = "../../../../study/jleague/motodata/"
dict_df = {}
for file in os.listdir(path):
    if file.find(".csv")==-1 or -1 < file.find("sample"): continue
    filename = os.path.splitext(file)[0]
    if   filename=="train_add":
        dict_df["train"]     = pd.concat([dict_df["train"],     pd.read_csv(path + file)]).reset_index(drop=True)
        dict_df["train"].loc[dict_df["train"].home=="ザスパ草津", "home"] = "ザスパクサツ群馬"
        dict_df["train"].loc[dict_df["train"].away=="ザスパ草津", "away"] = "ザスパクサツ群馬"
    elif filename=="condition_add":
        dict_df["condition"] = pd.concat([dict_df["condition"], pd.read_csv(path + file)]).reset_index(drop=True)
    elif filename=="test":
        dict_df["test"]      = pd.concat([dict_df["2014_add"],  pd.read_csv(path + file)]).reset_index(drop=True)
    else:
        dict_df[filename]    = pd.read_csv(path + file)

In [ ]:
for key, df in dict_df.items():
    print(key, df.shape)
    display(df.head(1))

In [ ]:
#クラブチームと都道府県
url  = "http://zatutisiki.com/7376.html"
soup = BeautifulSoup(requests.get(url).text, 'lxml')
list_pref_team = []
for row in soup.find_all("td"):
    #都道府県名
    if row.get("class")[0]=="column-1":
        pref = row.get_text()
    #チーム名
    if row.get("class")[0]=="column-2":
        if pref=="": continue
        list_pref_team.append([pref, row.get_text()])
df_pref_team = pd.DataFrame(list_pref_team, columns=["pref","home"])
df_pref_team.loc[df_pref_team.home=="ベカルタ仙台",          "home"] = "ベガルタ仙台"
df_pref_team.loc[df_pref_team.home=="北海道コンサドーレ札幌","home"] = "コンサドーレ札幌"

for pref in df_pref_team.pref.unique():
    for region, prefs in region_pref.items():
        if pref in prefs: df_pref_team.loc[df_pref_team.pref==pref, "region"] = region

In [ ]:
#チーム別の所在都道府県、スタジアム情報
df_stadium = dict_df['stadium']
df_stadium.rename(columns={"name":"stadium"}, inplace=True)
df_stadium.capa    = df_stadium.capa.astype(int)
df_stadium["pref"] = df_stadium.address.apply(lambda x: x[0:3])
df_stadium.loc[df_stadium.pref=="神奈川", "pref"] += "県"

df_team_info = pd.concat([dict_df["train"], dict_df["test"]])
df_team_info = df_team_info[["home","stadium"]].drop_duplicates()
df_team_info.groupby("home", as_index=False).stadium.count()
df_team_info["home_nor"] = df_team_info.home.apply(lambda x:unicodedata.normalize("NFKC", x))
df_team_info = pd.merge(df_team_info, df_pref_team, left_on="home_nor", right_on="home", how="outer", suffixes=("","_tmp"))
df_team_info.drop(["home_tmp","home_nor"], axis=1, inplace=True)
df_team_info.loc[df_team_info.home=="Ｖ・ファーレン長崎", "region"] = "九州"
df_team_info.loc[df_team_info.home=="Ｖ・ファーレン長崎", "pref"]   = "長崎県"
df_team_info = df_team_info.dropna()
df_team_info = pd.merge(df_team_info, df_stadium, on="stadium", suffixes=("_team","_stadium"))
df_team_info["local_stadium_opened"] = df_team_info.apply(lambda x: 1 if x["pref_team"]==x["pref_stadium"] else 0, axis=1)

In [ ]:
#同一地区、隣接地区
df_region = pd.concat([dict_df["train"], dict_df["test"]])
df_region = pd.merge(df_region, df_team_info, on=["home","stadium"])
df_region = pd.merge(df_region, df_team_info[["home","region"]].drop_duplicates(), left_on="away", right_on="home", suffixes=("","_away"))
df_region = df_region.assign(same_region = df_region.apply(lambda x: 1 if x.region==x.region_away else 0, axis=1),
                             near_region = df_region.apply(lambda x: 1 if x.region_away in near_region[x.region] else 0, axis=1))
df_region = df_region[["id","same_region","near_region"]]

In [ ]:
#2011年の観客動員数
def get_past_data(year):
    statime  = datetime.now()
    df_past  = pd.DataFrame()
    df_teams = pd.concat([dict_df['train'], dict_df['test']])
    df_teams["team_nor"] = df_teams.home.apply(lambda x: unicodedata.normalize("NFKC", x))
    df_teams = df_teams[["home","team_nor"]].drop_duplicates()
    for team, team_nor in zip(df_teams.home, df_teams.team_nor):
        search   = team_nor
        if team_nor=="東京ヴェルディ" : search = "東京ヴェルディ1969"
        if team_nor=="FC町田ゼルビア" : search = "町田ゼルビア"
        url      = "http://footballgeist.com/audience?id=team&no=" + search + "&season=" + str(year) + "#setubetu"
        soup     = BeautifulSoup(requests.get(url).text, 'lxml')
        row_team = soup.find_all("table")[2]
        if row_team.find("td") is None: continue

        team_info = []
        for row_team_td in row_team.find_all("td"):
            if row_team_td.find("a") is not None:
                href = row_team_td.find("a").get("href")
                if -1 < href.find("team"):
                    away_team = href.replace("team/","")
                    df_away   = df_teams.query("team_nor==@away_team")
                    if df_away.shape[0] != 0           : away_team = df_away.home.values[0]
                    if away_team=="東京ヴェルディ1969" : away_team = "東京ヴェルディ"
                    team_info.append(away_team)
                if -1 < href.find("match"):
                    match      = href.replace("match/","")
                    url        = "http://footballgeist.com/match/" + match
                    row_match  = BeautifulSoup(requests.get(url).text, 'lxml').find_all("table")[0]
                    row_match  = row_match.find_all("td")
                    stage      = np.nan
                    if row_match[0].get_text()=="J1": stage = "Ｊ１"
                    if row_match[0].get_text()=="J2": stage = "Ｊ２"
                    team_info += [stage, row_match[1].get_text(), row_match[6].get_text()]
            else:
                team_info.append(row_team_td.get_text())

        df      = pd.DataFrame(np.array(team_info).reshape(-1, 8),
                               columns=["section","away","stage","gameday","weather","y","ratio","stadium"]).assign(year=year, home=team)
        df_past = pd.concat([df_past, df])

    weekday = ["月","火","水","木","金","土","日"]
    df_past = df_past.assign(y          = df_past.y.astype(int),
                             section    = df_past.section.astype(int),
                             weekday    = df_past.gameday.apply(lambda x: weekday[pd.to_datetime(x.replace("年","/").replace("月","/").replace("日","")).weekday()]))
    df_past["y_ratio"] = df_past.ratio.apply(lambda x: x.replace("%","").replace("-","0")).astype(float) / 100
    df_past["y_ratio"] = np.log(df_past.y_ratio / (1 - df_past.y_ratio))
    print(datetime.now()-statime)
    return df_past

df_10 = get_past_data(2010)
df_11 = get_past_data(2011)

In [ ]:
#試合直前の順位
df_score = copy.deepcopy(dict_df['condition'])
df_score["home_win_point"] = df_score.apply(lambda x: 1 if x["home_score"]==x["away_score"] else 3 if x["home_score"] > x["away_score"] else 0, axis=1)
df_score["away_win_point"] = df_score.apply(lambda x: 1 if x["home_score"]==x["away_score"] else 0 if x["home_score"] > x["away_score"] else 3, axis=1)
df_train_test = pd.concat([dict_df['train'], dict_df['test']])
df_score      = pd.merge(df_train_test, df_score, on="id")
df_home_score = df_score[["id","year","gameday","stage","home","home_score","away_score","home_win_point"]]
df_home_score["home_away"] = 1
df_home_score.rename(columns={"home":"team", "home_win_point":"win_point", "home_score":"got_score", "away_score":"lost_score"}, inplace=True)
df_away_score = df_score[["id","year","gameday","stage","away","away_score","home_score","away_win_point"]]
df_away_score["home_away"] = 0
df_away_score.rename(columns={"away":"team", "away_win_point":"win_point", "away_score":"got_score", "home_score":"lost_score"}, inplace=True)
df_score_cnt  = pd.concat([df_home_score, df_away_score])
df_score_cnt["diff_score"] = df_score_cnt.got_score - df_score_cnt.lost_score
df_score_cnt  = df_score_cnt.sort_values(by=["year","team","id"], ascending=[True,True,True])

win_points, got_scores, diff_scores  = [], [], []
for year in df_score_cnt.year.unique():
    for team in df_score_cnt.team.unique():
        win_points  += list(df_score_cnt.query("year==@year & team==@team").win_point.cumsum())
        got_scores  += list(df_score_cnt.query("year==@year & team==@team").got_score.cumsum())
        diff_scores += list(df_score_cnt.query("year==@year & team==@team").diff_score.cumsum())
df_score_cnt = df_score_cnt.assign(win_point_sum  = win_points,
                                   got_score_sum  = got_scores,
                                   diff_score_sum = diff_scores)
ranked_team = []
for stage in df_score_cnt.stage.unique():
    for year in df_score_cnt.year.unique():
        df_all_team = df_score_cnt.query("year==@year & stage==@stage")[["team"]].drop_duplicates()
        for day in df_score_cnt.query("year==@year & stage==@stage").gameday.unique():
            df_rank = df_score_cnt.query("year==@year & gameday < @day & stage==@stage")
            df_rank = df_rank.groupby(["year","stage","team"], as_index=False).gameday.max()
            if df_rank.shape[0]==0:
                #その年初めての試合
                df_rank_wide = df_score_cnt.query("year==@year & gameday==@day & stage==@stage")
                df_rank_wide = df_rank_wide[["id"]].drop_duplicates().assign(ranking_home=1, ranking_away=1)
            else:
                #その年２回目以降の試合
                df_rank = pd.merge(df_all_team,  df_rank, on="team", how="left")
                df_rank = pd.merge(df_rank, df_score_cnt, on=["team","year","stage","gameday"], how="left")
                df_rank.fillna(0, inplace=True)
                df_rank.loc[df_rank.stage==0, "home_away"] = 2
                df_rank = df_rank.sort_values(by=["win_point_sum","got_score_sum","diff_score_sum"], ascending=[False,False,False]).reset_index(drop=False)
                df_rank = df_rank.assign(diff_other_team1 = df_rank.win_point_sum.diff(),
                                         diff_other_team2 = df_rank.got_score_sum.diff(),
                                         diff_other_team3 = df_rank.diff_score_sum.diff(),
                                         ranking          = [i+1 for i in range(df_rank.shape[0])])
                #勝ち点、総得点、得失点差が同じ場合はランキングを同じにする
                for _, row in df_rank.query("diff_other_team1==0 & diff_other_team2==0 & diff_other_team3==0").iterrows():
                    df_rank.loc[row.name, "ranking"] = df_rank.loc[row.name-1, "ranking"]

                df_rank_wide = pd.merge(df_score_cnt.query("gameday==@day"), df_rank[["team","ranking"]], on="team")
                df_rank_wide = pd.merge(df_rank_wide.query("home_away==1"),
                                        df_rank_wide.query("home_away==0"), on=["id","year","gameday","stage"], suffixes=("_home","_away"))
                df_rank_wide = df_rank_wide[["id","ranking_home","ranking_away"]]    
            ranked_team.append(df_rank_wide)
            
df_ranked_team = pd.concat(ranked_team)

In [ ]:
#対戦カード別過去の観客動員数
df_12_14  = pd.concat([dict_df['train'],dict_df['test']])
e_cols    = ["home","away","y"]
m_cols    = ["home","away"]
df_y_mean = pd.merge(df_10[e_cols], df_11[e_cols],                        on=m_cols, how="outer", suffixes=("_10","_11"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2012")[e_cols], on=m_cols, how="outer", suffixes=("_11","_12"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2013")[e_cols], on=m_cols, how="outer", suffixes=("_12","_13"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2014")[e_cols], on=m_cols, how="outer", suffixes=("_13","_14"))
df_y_mean.rename(columns={"y":"y_14"}, inplace=True)
e_cols    = ["home","stage"]
df_stage  = pd.merge(df_10[e_cols].drop_duplicates(), df_11[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_10","_11"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2012")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_11","_12"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2013")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_12","_13"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2014")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_13","_14"))
df_stage.rename(columns={"stage":"stage_14"}, inplace=True)
df_stage  = df_stage[df_stage.stage_14.notnull()].sort_values("home")

df_y_mean_long = pd.DataFrame()
for team, stage in zip(df_stage.home, df_stage.stage_14):
    df_mean_team  = copy.deepcopy(df_y_mean.query("home==@team"))
    df_stage_team = copy.deepcopy(df_stage.query("home==@team"))
    #最初のステージ
    if   0 < df_stage_team.query("stage_10==@stage").shape[0]:  year = 2010
    elif 0 < df_stage_team.query("stage_11==@stage").shape[0]:  year = 2011
    elif 0 < df_stage_team.query("stage_12==@stage").shape[0]:  year = 2012
    elif 0 < df_stage_team.query("stage_13==@stage").shape[0]:  year = 2013
    else:                                                       year = 2014

    mean_type="gmean"
    def mean_y(df_long, df_mean, year, cols, fill_col):
        df = copy.deepcopy(df_mean).assign(year = year)
        if   mean_type=="mean":   #算術平均
            df["y_mean"] = np.nanmean(np.array(df[cols]), axis=1)
        elif mean_type=="gmean":  #幾何平均
            df["y_mean"] = df.fillna(1).apply(lambda x: len(cols)-list(x[cols]).count(1), axis=1)
            df["y_mean"] = df.fillna(1).apply(lambda x: math.pow(reduce(operator.mul, list(x[cols])), 1 / x.y_mean if 0 < x.y_mean else 0), axis=1)
        df.y_mean.fillna(df[fill_col].mean(), inplace=True)
        return pd.concat([df_long, df[["home","away","year","y_mean"]]])
    
    if year != 2012 and 0 < df_stage_team.query("stage_12==@stage").shape[0]:
        df_y_mean_long = mean_y(df_y_mean_long, df_mean_team, 2012, ["y_10","y_11"], "y_11")
    if year != 2013 and 0 < df_stage_team.query("stage_13==@stage").shape[0]:
        df_y_mean_long = mean_y(df_y_mean_long, df_mean_team, 2013, ["y_10","y_11","y_12"], "y_12")
    if year != 2014 and 0 < df_stage_team.query("stage_14==@stage").shape[0]:
        df_y_mean_long = mean_y(df_y_mean_long, df_mean_team, 2014, ["y_10","y_11","y_12","y_13"], "y_12")
    if year == 2014:
        df_y_mean_long = mean_y(df_y_mean_long, df_mean_team, 2014, ["y_14"], "y_14")

In [ ]:
#対戦カード別CAGR
df_12_14  = pd.concat([dict_df['train'],dict_df['test']])
e_cols    = ["home","away","y"]
m_cols    = ["home","away"]
df_y_mean = pd.merge(df_10[e_cols], df_11[e_cols],                        on=m_cols, how="outer", suffixes=("_10","_11"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2012")[e_cols], on=m_cols, how="outer", suffixes=("_11","_12"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2013")[e_cols], on=m_cols, how="outer", suffixes=("_12","_13"))
df_y_mean = pd.merge(df_y_mean,     df_12_14.query("year==2014")[e_cols], on=m_cols, how="outer", suffixes=("_13","_14"))
df_y_mean.rename(columns={"y":"y_14"}, inplace=True)
e_cols    = ["home","stage"]
df_stage  = pd.merge(df_10[e_cols].drop_duplicates(), df_11[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_10","_11"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2012")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_11","_12"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2013")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_12","_13"))
df_stage  = pd.merge(df_stage, df_12_14.query("year==2014")[e_cols].drop_duplicates(), on="home", how="outer", suffixes=("_13","_14"))
df_stage.rename(columns={"stage":"stage_14"}, inplace=True)
df_stage  = df_stage[df_stage.stage_14.notnull()].sort_values("home")

def cagr(a, b, n):
    return ((b / a) ** (1 / n)) - 1
def check(x, cols_over, cols_0):
    return all(0 < y for y in x[cols_over]) and all(y==0 for y in x[cols_0])

df_y_mean.fillna(0, inplace=True)
df_y_mean["cagr_12"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_11, 2) if check(x, ["y_10","y_11"],        []) else 0, axis=1)
df_y_mean["cagr_13"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_12, 3) if check(x, ["y_10","y_11","y_12"], []) else 0, axis=1)
df_y_mean["cagr_13"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_11, 2) if check(x, ["y_10","y_11"],  ["y_12"]) else x.cagr_13, axis=1)
df_y_mean["cagr_13"] = df_y_mean.apply(lambda x: cagr(x.y_11, x.y_12, 2) if check(x, ["y_11","y_12"],  ["y_10"]) else x.cagr_13, axis=1)

df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_13, 4) if check(x, ["y_10","y_11","y_12","y_13"], []) else 0, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_13, 3) if check(x, ["y_10","y_12","y_13"],  ["y_11"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_12, 3) if check(x, ["y_10","y_11","y_12"],  ["y_13"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_13, 3) if check(x, ["y_10","y_11","y_13"],  ["y_12"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_11, x.y_13, 3) if check(x, ["y_11","y_12","y_13"],  ["y_10"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_12, x.y_13, 2) if check(x, ["y_12","y_13"],  ["y_10","y_11"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_11, x.y_13, 2) if check(x, ["y_11","y_13"],  ["y_10","y_12"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_11, x.y_12, 2) if check(x, ["y_11","y_12"],  ["y_10","y_13"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_13, 2) if check(x, ["y_10","y_13"],  ["y_11","y_12"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_12, 2) if check(x, ["y_10","y_12"],  ["y_11","y_13"]) else x.cagr_14, axis=1)
df_y_mean["cagr_14"] = df_y_mean.apply(lambda x: cagr(x.y_10, x.y_11, 2) if check(x, ["y_10","y_11"],  ["y_12","y_13"]) else x.cagr_14, axis=1)

df_y_mean_12   = df_y_mean[["home","away","cagr_12"]].assign(year=2012).rename(columns={"cagr_12":"cagr"})
df_y_mean_13   = df_y_mean[["home","away","cagr_13"]].assign(year=2013).rename(columns={"cagr_13":"cagr"})
df_y_mean_14   = df_y_mean[["home","away","cagr_14"]].assign(year=2014).rename(columns={"cagr_14":"cagr"})
df_y_cagr_long = pd.concat([df_y_mean_12, df_y_mean_13, df_y_mean_14])

In [ ]:
#試合前連勝記録
df = copy.deepcopy(pd.concat([dict_df['train'], dict_df['test']]))
df = pd.merge(df, dict_df["condition"], on="id")
df = df[["id","home","away","stage","year","gameday","home_score","away_score"]]

df_series_win = pd.DataFrame()
for year in df.year.unique():
    for team in df.query("year==@year").home.unique():
        df_win = copy.deepcopy(df.query("year==@year & (home==@team | away==@team)"))
        df_win["win"] = 0
        df_win.loc[(df_win.home==team)&(df_win.away_score <= df_win.home_score), "win"] = 1
        df_win.loc[(df_win.away==team)&(df_win.home_score <= df_win.away_score), "win"] = 1
        point, series_points = -1, []
        for _, row in df_win.iterrows():
            if row.home==team:
                home_away = "home"
            else:
                home_away = "away"
            if row.win==1:
                point += 1
                series_points.append([home_away, point])
            else:
                if -1 < point:
                    series_points.append([home_away, point + 1])
                else:
                    series_points.append([home_away, 0])
                point  = -1
        df_win = df_win.assign(home_away    = np.array(series_points)[:,0],
                               series_point = np.array(series_points)[:,1])
        df_series_win = pd.concat([df_series_win, df_win])
        
df_home = df_series_win.query("home_away=='home'")
df_away = df_series_win.query("home_away=='away'")[["id","series_point"]]
df_series_win = pd.merge(df_home, df_away, on="id", suffixes=("_home","_away"))[["id","series_point_home","series_point_away"]]
df_series_win.series_point_away = df_series_win.series_point_away.astype(int)
df_series_win.series_point_home = df_series_win.series_point_home.astype(int)

In [ ]:
path     = "../../../../study/jleague/team/"
df_wcup  = pd.concat([pd.read_csv(path + "wcup_train.csv"),
                      pd.read_csv(path + "wcup_train_add.csv"),
                      pd.read_csv(path + "wcup_test.csv")])
df_money = pd.concat([pd.read_csv(path + "money_train.csv"),
                      pd.read_csv(path + "money_train_add.csv"),
                      pd.read_csv(path + "money_test.csv")])

In [ ]:
def add_cols(df):
    df_add = copy.deepcopy(df)
    df_add = df_add.assign(weekday    = df_add.gameday.apply(lambda x: x[x.find("(")+1:x.find("(")+2]),
                           section    = df_add.match.apply(lambda x: x[1:x.find("節")]).astype(int),
                           time_split = df_add.time.apply(lambda x: x[:2]).astype(int),
                           tv_num     = df_add.tv.apply(lambda x: len(x.split("／"))).astype(int),
                           champ      = 0)
    #前年優勝、準優勝チーム
    df_add.loc[(df_add.year==2012)&(df_add.home.isin(["柏レイソル","名古屋グランパス"])),         "champ"] = 1
    df_add.loc[(df_add.year==2013)&(df_add.home.isin(["サンフレッチェ広島","ベガルタ仙台"])),     "champ"] = 1
    df_add.loc[(df_add.year==2014)&(df_add.home.isin(["サンフレッチェ広島","横浜Ｆ・マリノス"])), "champ"] = 1
    #天気、気温
    df_add = pd.merge(df_add, dict_df["condition"][["id","weather","temperature"]], on="id")
    #スタジアム収容人数、地区
    df_add = pd.merge(df_add, df_team_info,   on=["home","stadium"])
    #同一地区、隣接地区
    df_add = pd.merge(df_add, df_region,      on="id")
    #試合前ランキング
    df_add = pd.merge(df_add, df_ranked_team, on="id")
    #試合前連勝記録
    df_add = pd.merge(df_add, df_series_win,  on="id")
    #world cup出場選手数
    df_add = pd.merge(df_add, df_wcup,        on="id")
    #出場選手の年棒
    df_add = pd.merge(df_add, df_money,       on="id")
    #同一対戦カードCAGR
    df_add = pd.merge(df_add, df_y_cagr_long, on=["home","away","year"])
    #同一対戦カード観客動員数平均
    df_add = pd.merge(df_add, df_y_mean_long, on=["home","away","year"])
    #最終節、優勝決定戦
    df_add = df_add.assign(section_last    = df_add.apply(lambda x: 1 if (x.stage=="Ｊ１" and x.section in [33,34]) or (x.stage=="Ｊ２" and x.section in [41,42]) else 0, axis=1),
                           champ_candidate = 0)
    df_add.loc[(df_add.stage=="Ｊ１")&(df_add.section.isin([33,34]))&(df_add.ranking_home < 5), "champ_candidate"] = 1
    df_add.loc[(df_add.stage=="Ｊ２")&(df_add.section.isin([41,42]))&(df_add.ranking_home < 5), "champ_candidate"] = 1
    #晴、雨に統一
    df_add = re_weather(df_add)
    #平日、休日に統一
    df_add = re_weekday(df_add)
    return df_add

# train
except_teams = ['ザスパ草津','ＦＣ町田ゼルビア','ガイナーレ鳥取']
df_train     = dict_df["train"].query("y != 0 & home not in @except_teams & away not in @except_teams")
df_train     = add_cols(df_train)
# test
except_ids = dict_df['2014_add'].id
df_test    = dict_df["test"].query("id not in @except_ids")
df_test    = add_cols(df_test)

df_train.shape, df_test.shape

In [ ]:
will_drop_cols  = ["year","match","gameday","time","home","away","stadium","tv",
                   "address","pref_team","pref_stadium","champ_candidate","local_stadium_opened","champ","cagr"]
std_except_cols = [col for col in df_train if len(df_train[col].unique()) < 3] + ["id","y","y_mean","cagr"]

std_cols        = list(df_train.select_dtypes(include=["int","float"]).columns)
std_cols        = [col for col in std_cols     if col not in will_drop_cols + std_except_cols]
dummies_cols    = list(df_train.select_dtypes(include="object").columns)
dummies_cols    = [col for col in dummies_cols if col not in will_drop_cols + std_cols]

# 標準化
df_train, df_test = standardization(df_train, std_cols, df_test)

# ダミー変数化
df_dummies   = pd.concat([df_train, df_test])
df_dummies   = dummies(df_dummies, dummies_cols)
df_train, df_test = df_dummies[df_dummies.y.notnull()], df_dummies[df_dummies.y.isnull()]
df_train.y   = df_train.y.astype(int)
df_test.drop("y", axis=1, inplace=True)

In [ ]:
is_ratio = False
#観客動員数の割合
if is_ratio:
    df_train.y = np.log((df_train.y / df_train.capa) / (1 - (df_train.y / df_train.capa)))
x = df_train.drop(["id","y"] + std_cols + will_drop_cols, axis=1)
y = df_train.y
train_x = copy.deepcopy(df_train.query("(year == 2013 & '08/01' <= gameday) | year in (2012,2014)"))
train_x.drop(["id","y"] + std_cols + will_drop_cols, axis=1, inplace=True)
train_y = copy.deepcopy(df_train.query("(year == 2013 & '08/01' <= gameday) | year in (2012,2014)")).y
valid_x = copy.deepcopy(df_train.query("year == 2013 & gameday <= '08/01'"))
valid_x.drop(["id","y"] + std_cols + will_drop_cols, axis=1, inplace=True)
valid_y = copy.deepcopy(df_train.query("year == 2013 & gameday <= '08/01'")).y
x_test  = df_test.drop(["id"] + std_cols + will_drop_cols, axis=1)

print(x.shape)
x.head(2)

In [ ]:
seed = 15
#train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=seed)

#classifier = SVR(kernel='rbf', gamma=1/2, C=1.0)
classifier = xgb.XGBRegressor(gamma=0, learning_rate=0.2, max_depth=3, min_child_weight=20, n_estimators=100, random_state=seed)
classifier.fit(x, y)

print(np.sqrt(mean_squared_error(classifier.predict(x), y)))
print(np.sqrt(mean_squared_error(classifier.predict(train_x), train_y)))
print(np.sqrt(mean_squared_error(classifier.predict(valid_x), valid_y)))

In [ ]:
df_train["pred"] = classifier.predict(x)
df_train["redi"] = df_train.y - df_train.pred
df_train[["id","home","away","year","section","y_mean","y","pred","redi"]].sort_values("redi")

In [ ]:
pred = classifier.predict(x_test)
#pred = 1 / (1 + np.exp(-pred)) * df_test.capa

In [ ]:
path = "../../../../study/jleague/submit/"
df_result = pd.DataFrame({"id" : df_test.id, "result" : pred})
df_result.to_csv(path + "20180807_5.csv", index=False, header=None)

In [ ]:
df_12_14 = copy.deepcopy(pd.concat([dict_df['train'],dict_df['test']]))
df_12_14 = pd.merge(df_12_14, dict_df['condition'], on="id")
df_12_14 = df_12_14.assign(section = df_12_14.match.apply(lambda x: x[1:x.find("節")]).astype(int),
                           weekday = df_12_14.gameday.apply(lambda x: x[x.find("(")+1:x.find("(")+2]))
df_12_14 = re_weather(df_12_14)
df_12_14 = re_weekday(df_12_14)
df_12_14["info"] = df_12_14.weekday + "-" + df_12_14.weather + "-" + df_12_14.section.astype(str)

df_11_info = copy.deepcopy(df_11[["home","away","stage","y","section","gameday","weekday","weather"]])
df_11_info = re_weather(df_11_info)
df_11_info = re_weekday(df_11_info)
df_11_info["info"] = df_11_info.weekday + "-" + df_11_info.weather + "-" + df_11_info.section.astype(str)

df_12_info = df_12_14.query("year==2012")[["home","away","stage","y","info"]]
df_13_info = df_12_14.query("year==2013")[["home","away","stage","y","info"]]
df_14_info = df_12_14.query("year==2014")[["home","away","stage","y","info"]]

df_11_mean = df_11[["home","y"]].groupby("home", as_index=False).y.mean()
df_12_mean = df_12_14.query("year==2012")[["home","y"]].groupby("home", as_index=False).y.mean()
df_13_mean = df_12_14.query("year==2013")[["home","y"]].groupby("home", as_index=False).y.mean()
df_14_mean = df_12_14.query("year==2014")[["home","y"]].groupby("home", as_index=False).y.mean()
df_11_mean.rename(columns={"y":"y_mean_11"}, inplace=True)
df_12_mean.rename(columns={"y":"y_mean_12"}, inplace=True)
df_13_mean.rename(columns={"y":"y_mean_13"}, inplace=True)
df_14_mean.rename(columns={"y":"y_mean_14"}, inplace=True)

df_11_12 = pd.merge(df_11_info, df_12_info, on=["home","away"], how="outer", suffixes=("_11","_12"))
df_11_13 = pd.merge(df_11_12,   df_13_info, on=["home","away"], how="outer", suffixes=("_12","_13"))
df_11_14 = pd.merge(df_11_13,   df_14_info, on=["home","away"], how="outer", suffixes=("_13","_14")).sort_values(by=["home","away"])
df_11_14 = pd.merge(df_11_14,   df_11_mean, on= "home",         how="left")
df_11_14 = pd.merge(df_11_14,   df_12_mean, on= "home",         how="left")
df_11_14 = pd.merge(df_11_14,   df_13_mean, on= "home",         how="left")
df_11_14 = pd.merge(df_11_14,   df_14_mean, on= "home",         how="left")

In [ ]:
df_11_14[["home","away","y_11","info_11","y_12","info_12","y_13","info_13","y_14","info_14"]]